In [1]:
print("STARTO")
import time
import os
start_time = time.time()

using_gpu = False
if not using_gpu:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5"

from lib.detector.YOLO.train.trainer import Trainer
from lib.detector.YOLO import create_detector_model_from_trained
from lib.detector.YOLO import create_training_model_from_detector
from lib.detector.YOLO import Yolo

STARTO


Using TensorFlow backend.


In [2]:
# raise ValueError("test")

In [3]:
from keras.models import load_model
our_ver = load_model('./model_data/yolo.h5')

/src/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
labels = [
    'person', 'bicycle', 'car', 'motorbike', 'airplane', 'bus', 'train',
    'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
    'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
    'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
    'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant',
    'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
    'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
]
our_yolo = Yolo(labels=labels, yolo_model=our_ver)

In [5]:
import numpy as np
import json
with open('./train_val_config/ua_voc_4.json', 'r') as fp:
    data = json.load(fp)
    train_imgs = data['train']
    valid_imgs = data['val']

In [6]:
import cv2
# "boxes":[], "scores":[]
labels = ['bicycle','bus','car','motorbike']
tests = valid_imgs
gt_boxes = {label:{test['filename']:[] for test in tests} for label in labels}
pred_boxes = {label:{test['filename']:{} for test in tests} for label in labels}
from pprint import pprint

for _dat in valid_imgs:
    key = _dat['filename']
    for obj in _dat['object']:
        gt_boxes[obj['name']][key].append([obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax']])

    _img = cv2.imread(key)
    rgb_img = cv2.cvtColor(_img, cv2.COLOR_BGR2RGB)
#     obj['ymin'], obj['xmin'], obj['ymax'], obj['xmax']
    results = our_yolo.get_localization(rgb_img, 
                              result_mapping={
                                  1: 'bicycle',
                                  5: 'bus',
                                  2: 'car',
                                  3: 'motorbike'
                              },
                              score_threshold=0.25,
                              class_threshold={
                                  'bicycle': 0.25,
                                  'bus': 0.25,
                                  'car': 0.25,
                                  'motorbike': 0.25
                              },
                              special_mode=False
                            )
    for cls_name in results:
        pred_boxes[cls_name][key]['boxes'] = []
        for box in results[cls_name]['boxes']:
            pred_boxes[cls_name][key]['boxes'].append([box[1], box[0], box[3], box[2]])
        pred_boxes[cls_name][key]['scores'] = results[cls_name]['scores']

In [7]:
# gt_boxes = {label:{test['filename']:[] for test in tests} for label in labels}
# pred_boxes = {label:{test['filename']:{} for test in tests} for label in labels}
import json

def myconverter(o):
    if isinstance(o, np.float32):
        return float(o)
    
with open('gt_boxes_yolo.json', 'w') as outfile:
    json.dump(gt_boxes, outfile, default=myconverter)
with open('pred_boxes_yolo.json', 'w') as outfile:
    json.dump(pred_boxes, outfile, default=myconverter)

In [8]:
!mv gt_boxes_yolo.json analysis/gt_boxes_yolo.json
!mv pred_boxes_yolo.json analysis/pred_boxes_yolo.json